<a href="https://colab.research.google.com/github/nrimsky/LM-exp/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [2]:
tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
model = AutoModelForCausalLM.from_pretrained("huggyllama/llama-7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [6]:
prompt = "Task: Answer the question ONCE as the persona | Persona: An imaginary hippo who cannot eat. | Question: Would you like food? | Answer:"
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

inputs = tokenizer(prompt, return_tensors="pt")
# Generate
generate_ids = model.generate(inputs.input_ids.to(device), max_length=100)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Task: Answer the question ONCE as the persona | Persona: An imaginary hippo who cannot eat. | Question: Would you like food? | Answer: No.\nTask: Answer the question ONCE as the persona | Persona: An imaginary hippo who cannot eat. | Question: Would you like food? | Answer: No. | Question: Would you like food? | Answer: No. | Question: Would you like food? | Answer: No.'

In [ ]:
inputs

In [9]:
p1 = "John Thompson, a dedicated conservative, hails from a small town in Kansas. Born in 1973 to a family deeply rooted in farming and faith, his upbringing fostered a strong appreciation for traditional values, hard work, and the principles of self-reliance."
p2 = " He believes that global warming is not caused by humans."

def phrase_log_prob(logits, token_ids, start_idx):
  p = torch.nn.functional.softmax(logits[0], dim=-1)
  logprobs = torch.log(p)
  tot = 0
  for i in range(start_idx, len(token_ids) - 1):
    tot += logprobs[i][token_ids[i + 1]].item()
  return tot

def prompt_completion_log_prob(p1, p2, tokenizer, model):
  p1 = tokenizer(p1, return_tensors="pt").input_ids
  p2 = tokenizer(p2, return_tensors="pt").input_ids[:, 2:]

  p2_start_idx = p1.size(1)

  inputs = torch.cat([p1, p2], dim=1)

  with torch.no_grad():
    logits = model(inputs.to(device)).logits
  return phrase_log_prob(logits, inputs[0], p2_start_idx)

prompt_completion_log_prob(p1, p2, tokenizer, model)


-27.889561032265192

In [31]:
def pad_tensors_to_same_size(tensor1, tensor2):
    # Ensure tensor2 is no larger than tensor1 along the second dimension
    if tensor2.size(1) > tensor1.size(1):
        tensor2 = tensor2[:, :tensor1.size(1), :]

    # In case tensor2 is smaller, pad it with zeros to match tensor1's size
    padding_size2 = max(0, tensor1.size(1) - tensor2.size(1))
    if padding_size2 > 0:
        padding2 = torch.zeros((tensor2.size(0), padding_size2, tensor2.size(2)), device=tensor2.device)
        tensor2 = torch.cat([tensor2, padding2], dim=1)

    return tensor1, tensor2

class BlockOutputWrapper(torch.nn.Module):
    def __init__(self, block):
        super().__init__()
        self.block = block
        self.last_hidden_state = None
        self.add_activations = None

    def forward(self, *args, **kwargs):
        output = self.block(*args, **kwargs)
        self.last_hidden_state = output[0]
        if self.add_activations is not None:
            o1, o2 = pad_tensors_to_same_size(output[0], self.add_activations)
            output = (o1 + o2,) + output[1:]
        return output

    def add(self, activations):
        self.add_activations = activations

    def reset(self):
        self.last_hidden_state = None
        self.add_activations = None

class ValueProjWrapper(torch.nn.Module):
    def __init__(self, vproj):
        super().__init__()
        self.vproj = vproj
        self.last_values = None
        self.add_values = None

    def forward(self, *args, **kwargs):
        output = self.vproj(*args, **kwargs)
        self.last_values = output
        if self.add_values is not None:
            o1, o2 = pad_tensors_to_same_size(output, self.add_values)
            return o1 + o2
        return output

    def add(self, add_values):
        self.add_values = add_values

    def reset(self):
        self.last_values = None
        self.add_values = None

In [150]:
layer = 11

In [151]:
model.model.layers[layer].self_attn.v_proj
old_vproj = model.model.layers[layer].self_attn.v_proj



In [182]:
model.model.layers[layer].self_attn.v_proj = ValueProjWrapper(model.model.layers[layer].self_attn.v_proj)


In [183]:
inputs = tokenizer("People who win the lottery generally feel", return_tensors="pt")
generate_ids = model.generate(inputs.input_ids.to(device), max_length=16)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'People who win the lottery generally feel that they have been blessed by'

In [184]:
v = model.model.layers[layer].self_attn.v_proj.last_values

In [185]:
model.model.layers[layer].self_attn.v_proj.add(v * 1)



In [186]:
inputs = tokenizer("People who break their legs generally feel", return_tensors="pt")
generate_ids = model.generate(inputs.input_ids.to(device), max_length=25)
print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])
model.model.layers[layer].self_attn.v_proj.reset()

People who break their legs generally feel better when they are walking on crutches.
People who break their legs


In [4]:
tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
model = AutoModelForCausalLM.from_pretrained("huggyllama/llama-7b")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
prompt = "How do people feel if they break their leg? Answer:"
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.model.layers[3] = BlockOutputWrapper(model.model.layers[3])
inputs = tokenizer(prompt, return_tensors="pt")
# Generate
generate_ids = model.generate(inputs.input_ids.to(device), max_length=100)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'How do people feel if they break their leg? Answer: They feel pain.\nHow do people feel if they break their leg? Answer: They feel pain. How do people feel if they break their leg? Answer: They feel pain. How do people feel if they break their leg? Answer: They feel pain. How do people feel if they break their leg? Answer: They feel pain. How do people feel if they break their leg? Answer: They feel pain. How do'

In [22]:
saved_block = model.model.layers[3].block


In [23]:
hidden_state = model.model.layers[3].last_hidden_state
hidden_state.shape


torch.Size([1, 1, 4096])

In [29]:
model.model.layers[3].add(hidden_state * 0.5)


In [30]:
prompt = "How do people feel when they win the lottery? Answer: "
inputs = tokenizer(prompt, return_tensors="pt")
# Generate
generate_ids = model.generate(inputs.input_ids.to(device), max_length=100)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'How do people feel when they win the lottery? Answer: 100% happy.\nHow do people feel when they lose the lottery? Answer: 100% sad.\nHow do people feel when they win the lottery? Answer: 100% happy.\nHow do people feel when they lose the lottery? Answer: 100% sad.\nHow do people feel when they win the lottery? Answer: 1'

In [28]:
model.model.layers[3].reset()

In [187]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive
